In [1]:
# Check if the easydiffraction library is installed.
# If not, install it with the 'visualization' extras.
# Needed when running remotely (e.g. Colab) where the lib is absent.
import builtins
import importlib.util

if (hasattr(builtins, '__IPYTHON__') and
    importlib.util.find_spec('easydiffraction') is None):
    !pip install 'easydiffraction[visualization]'

# Structure Refinement: PbSO4, NPD + XRD

This example demonstrates a more advanced use of the EasyDiffraction
library by explicitly creating and configuring sample models and
experiments before adding them to a project. It could be more suitable
for users who are interested in creating custom workflows. This
tutorial provides minimal explanation and is intended for users
already familiar with EasyDiffraction.

The tutorial covers a Rietveld refinement of PbSO4 crystal structure
based on the joint fit of both X-ray and neutron diffraction data.

## Import Library

In [2]:
from easydiffraction import ExperimentFactory
from easydiffraction import Project
from easydiffraction import SampleModelFactory
from easydiffraction import download_data

## Define Sample Model

This section shows how to add sample models and modify their
parameters.

#### Create Sample Model

In [3]:
model = SampleModelFactory.create(name='pbso4')

#### Set Space Group

In [4]:
model.space_group.name_h_m = 'P n m a'

#### Set Unit Cell

In [5]:
model.cell.length_a = 8.47
model.cell.length_b = 5.39
model.cell.length_c = 6.95

#### Set Atom Sites

In [6]:
model.atom_sites.add(
    label='Pb',
    type_symbol='Pb',
    fract_x=0.1876,
    fract_y=0.25,
    fract_z=0.167,
    wyckoff_letter='c',
    b_iso=1.37,
)
model.atom_sites.add(
    label='S',
    type_symbol='S',
    fract_x=0.0654,
    fract_y=0.25,
    fract_z=0.684,
    wyckoff_letter='c',
    b_iso=0.3777,
)
model.atom_sites.add(
    label='O1',
    type_symbol='O',
    fract_x=0.9082,
    fract_y=0.25,
    fract_z=0.5954,
    wyckoff_letter='c',
    b_iso=1.9764,
)
model.atom_sites.add(
    label='O2',
    type_symbol='O',
    fract_x=0.1935,
    fract_y=0.25,
    fract_z=0.5432,
    wyckoff_letter='c',
    b_iso=1.4456,
)
model.atom_sites.add(
    label='O3',
    type_symbol='O',
    fract_x=0.0811,
    fract_y=0.0272,
    fract_z=0.8086,
    wyckoff_letter='d',
    b_iso=1.2822,
)

## Define Experiments

This section shows how to add experiments, configure their parameters,
and link the sample models defined in the previous step.

### Experiment 1: npd

#### Download Data

In [7]:
data_path1 = download_data(id=13, destination='data')

Getting data...


Data #13: PbSO4, D1A (ILL)


✅ Data #13 downloaded to 'data/ed-13.dat'


#### Create Experiment

In [8]:
expt1 = ExperimentFactory.create(
    name='npd',
    data_path=data_path1,
    radiation_probe='neutron',
)

Data loaded successfully


Experiment 🔬 'npd'. Number of data points: 1801


#### Set Instrument

In [9]:
expt1.instrument.setup_wavelength = 1.91
expt1.instrument.calib_twotheta_offset = -0.1406

#### Set Peak Profile

In [10]:
expt1.peak.broad_gauss_u = 0.139
expt1.peak.broad_gauss_v = -0.412
expt1.peak.broad_gauss_w = 0.386
expt1.peak.broad_lorentz_x = 0
expt1.peak.broad_lorentz_y = 0.088

#### Set Background

Select the background type.

In [11]:
expt1.background_type = 'line-segment'

Background type for experiment 'npd' changed to


line-segment


Add background points.

In [12]:
for id, x, y in [
    ('1', 11.0, 206.1624),
    ('2', 15.0, 194.75),
    ('3', 20.0, 194.505),
    ('4', 30.0, 188.4375),
    ('5', 50.0, 207.7633),
    ('6', 70.0, 201.7002),
    ('7', 120.0, 244.4525),
    ('8', 153.0, 226.0595),
]:
    expt1.background.add(id=id, x=x, y=y)

#### Set Linked Phases

In [13]:
expt1.linked_phases.add(id='pbso4', scale=1.5)

### Experiment 2: xrd

#### Download Data

In [14]:
data_path2 = download_data(id=16, destination='data')

Getting data...


Data #16: PbSO4, laboratory X-ray


✅ Data #16 downloaded to 'data/ed-16.dat'


#### Create Experiment

In [15]:
expt2 = ExperimentFactory.create(
    name='xrd',
    data_path=data_path2,
    radiation_probe='xray',
)

Data loaded successfully


Experiment 🔬 'xrd'. Number of data points: 3601


#### Set Instrument

In [16]:
expt2.instrument.setup_wavelength = 1.540567
expt2.instrument.calib_twotheta_offset = -0.05181

#### Set Peak Profile

In [17]:
expt2.peak.broad_gauss_u = 0.304138
expt2.peak.broad_gauss_v = -0.112622
expt2.peak.broad_gauss_w = 0.021272
expt2.peak.broad_lorentz_x = 0
expt2.peak.broad_lorentz_y = 0.057691

#### Set Background

Select background type.

In [18]:
expt2.background_type = 'chebyshev polynomial'

Background type for experiment 'xrd' changed to


chebyshev polynomial


Add background points.

In [19]:
for id, x, y in [
    ('1', 0, 119.195),
    ('2', 1, 6.221),
    ('3', 2, -45.725),
    ('4', 3, 8.119),
    ('5', 4, 54.552),
    ('6', 5, -20.661),
]:
    expt2.background.add(id=id, order=x, coef=y)

#### Set Linked Phases

In [20]:
expt2.linked_phases.add(id='pbso4', scale=0.001)

## Define Project

The project object is used to manage sample models, experiments, and
analysis.

#### Create Project

In [21]:
project = Project()

#### Add Sample Model

In [22]:
project.sample_models.add(sample_model=model)

#### Add Experiments

In [23]:
project.experiments.add(experiment=expt1)
project.experiments.add(experiment=expt2)

## Perform Analysis

This section outlines the analysis process, including how to configure
calculation and fitting engines.

#### Set Calculator

In [24]:
project.analysis.current_calculator = 'cryspy'

Current calculator changed to


cryspy


#### Set Fit Mode

In [25]:
project.analysis.fit_mode = 'joint'

Current fit mode changed to


joint


#### Set Minimizer

In [26]:
project.analysis.current_minimizer = 'lmfit (leastsq)'

Current minimizer changed to


lmfit (leastsq)


#### Set Fitting Parameters

Set sample model parameters to be optimized.

In [27]:
model.cell.length_a.free = True
model.cell.length_b.free = True
model.cell.length_c.free = True

Set experiment parameters to be optimized.

In [28]:
expt1.linked_phases['pbso4'].scale.free = True

expt1.instrument.calib_twotheta_offset.free = True

expt1.peak.broad_gauss_u.free = True
expt1.peak.broad_gauss_v.free = True
expt1.peak.broad_gauss_w.free = True
expt1.peak.broad_lorentz_y.free = True

In [29]:
expt2.linked_phases['pbso4'].scale.free = True

expt2.instrument.calib_twotheta_offset.free = True

expt2.peak.broad_gauss_u.free = True
expt2.peak.broad_gauss_v.free = True
expt2.peak.broad_gauss_w.free = True
expt2.peak.broad_lorentz_y.free = True

for term in expt2.background:
    term.coef.free = True

#### Perform Fit

In [30]:
project.analysis.fit()
project.analysis.show_fit_results()

Using all experiments 🔬 ['npd', 'xrd'] for 'joint' fitting


🚀 Starting fit process with 'lmfit (leastsq)'...


📈 Goodness-of-fit (reduced χ²) change:


,iteration,χ²,improvement [%]
1,1,37.01,
2,25,16.30,56.0% ↓
3,136,16.21,


<IPython.core.display.Javascript object>

...

📏 Weighted R-factor (wR): 17.09%


📈 Fitted parameters:


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,datablock,category,entry,parameter,start,fitted,uncertainty,units,change
1,pbso4,cell,,length_a,8.4700,8.4693,0.0002,Å,0.01 % ↓
2,pbso4,cell,,length_b,5.3900,5.3912,0.0001,Å,0.02 % ↑
3,pbso4,cell,,length_c,6.9500,6.9504,0.0002,Å,0.01 % ↑
4,npd,linked_phases,pbso4,scale,1.5000,1.4622,0.0049,,2.52 % ↓
5,npd,peak,,broad_gauss_u,0.1390,0.2901,0.0350,deg²,108.67 % ↑
6,npd,peak,,broad_gauss_v,-0.4120,-0.6390,0.0531,deg²,55.10 % ↑
7,npd,peak,,broad_gauss_w,0.3860,0.4552,0.0186,deg²,17.92 % ↑
8,npd,peak,,broad_lorentz_y,0.0880,0.0931,0.0034,deg,5.76 % ↑
9,npd,instrument,,twotheta_offset,-0.1406,-0.1385,0.0019,deg,1.47 % ↓
10,xrd,linked_phases,pbso4,scale,0.0010,0.0010,0.0000,,1.52 % ↓


#### Plot Measured vs Calculated

In [31]:
project.plot_meas_vs_calc(expt_name='npd', x_min=35.5, x_max=38.3, show_residual=True)

In [32]:
project.plot_meas_vs_calc(expt_name='xrd', x_min=29.0, x_max=30.4, show_residual=True)